# Kết nối với drive

In [36]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
import re, string
import nltk
from sklearn.metrics import roc_curve, auc

import tensorflow as tf
import tensorflow.keras.layers as L
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import SparseCategoricalCrossentropy

## Load dữ liệu

In [38]:
df=pd.read_csv('/content/drive/MyDrive/NLP_Food_review_classifier/full_train.csv')

df_test=pd.read_csv('/content/drive/MyDrive/NLP_Food_review_classifier/test.csv')

In [39]:
df.head()

,Unnamed: 0,RevId,UserId,Comment,image_urls,Rating
0,0,3839333,10106093.0,"Xôi dẻo, đồ ăn đậm vị. Hộp xôi được lót lá trô...",['https://images.foody.vn/res/g97/966781/s800/...,1.0
1,1,2824877,786914.0,Gọi ship 1 xuất cari gà bánh naan và 3 miếng g...,['https://images.foody.vn/res/g69/688413/s800/...,0.0
2,2,9816702,22467889.0,"Thời tiết lạnh như này, cả nhà rủ nhau đến leg...",['https://images.foody.vn/res/g72/715078/s800/...,1.0
3,3,2684585,1889449.0,Em có đọc review thấy mng bảo trà sữa nướng đề...,['https://images.foody.vn/res/g90/895545/s800/...,0.0
4,4,2737987,8839942.0,"Đồ ăn rất ngon, nhà hàng cũng rất đẹp, tất cả ...",['https://images.foody.vn/res/g4/30186/s800/fo...,1.0


In [40]:
df_test.head()

,Unnamed: 0,RevId,UserId,Comment,image_urls
0,0,781115,1326532,Trà táo 35k\nCookie socola 38k \nNước ở đây bì...,['https://images.foody.vn/res/g8/73091/s800/fo...
1,1,1219481,422306,Hôm rồi trung tâm mình tổ chức noel party ở đâ...,['https://images.foody.vn/res/g1/33/s800/foody...
2,2,1703765,9779143,Thịt gà của quán là nhất đấy. Đi ăn gọi liền 4...,['https://images.foody.vn/res/g66/659655/s800/...
3,3,4870346,12924388,Hai đứa ăn xong đau bụng cả ngày\nChân gà ok n...,['https://images.foody.vn/res/g78/772244/s800/...
4,4,2638711,1134279,Mình vừa thử trưa nay. Điểm cộng đầu tiên là b...,['https://images.foody.vn/res/g74/737874/s800/...


## Xử lý dữ liệu

Do nhãn ở df ít hơn comment => xóa các bản ghi thiếu nhãn

Do ở tập df_test có một số bản ghi không có comment => điền thêm

In [41]:
df = df.dropna()  # xóa nan
df_test = df_test.fillna(value = "Tôi không có bình luận gì")  # điền vào các giá trị khuyết thiếu

In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9070 entries, 0 to 9072
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  9070 non-null   object 
 1   RevId       9070 non-null   object 
 2   UserId      9070 non-null   float64
 3   Comment     9070 non-null   object 
 4   image_urls  9070 non-null   object 
 5   Rating      9070 non-null   float64
dtypes: float64(2), object(4)
memory usage: 496.0+ KB


In [43]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5103 entries, 0 to 5102
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5103 non-null   int64 
 1   RevId       5103 non-null   int64 
 2   UserId      5103 non-null   int64 
 3   Comment     5103 non-null   object
 4   image_urls  5103 non-null   object
dtypes: int64(3), object(2)
memory usage: 199.5+ KB


## Tiền huấn luyện mô hình

In [44]:
def preprocess(text):  # Hàm xử lý, làm sạch data
    text = str(text)
    try:
      text = text.lower() 
    except:
      text = text
    text=text.strip() 
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text) 
    text = re.sub('\s+', ' ', text) 
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text) 

    return text

In [45]:
tweet =" Nhà _ hàng này : ngon quá"
preprocess(tweet)

'nhà hàng này ngon quá'

In [46]:
# Tiến hành làm sạch

df['clean_text'] = df['Comment'].apply(lambda x: preprocess(x))

In [47]:
# Chia dữ liệu

comment = df['clean_text'].values
y = df['Rating'].values

comment_train, comment_test, y_train, y_test = train_test_split(comment, y, test_size=0.25, random_state=42)

## Tạo Word Embedding

In [48]:
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(comment_train)

X_train = tokenizer.texts_to_sequences(comment_train)
X_test = tokenizer.texts_to_sequences(comment_test)

vocab_size = len(tokenizer.word_index) + 1 

EMBEDDING_DIMS = 16

## Thêm padding => cố định kích thước đầu vào

In [49]:
maxlen = 150

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

## Xây dựng mô hình

In [52]:
model=tf.keras.Sequential()

model.add(L.Embedding(vocab_size, EMBEDDING_DIMS, input_length=maxlen))
model.add(L.Bidirectional(L.LSTM(256,return_sequences=True)))
model.add(L.Bidirectional(L.LSTM(256,return_sequences=True)))
model.add(L.GlobalMaxPool1D())
model.add(L.Dropout(0.5))
model.add(L.Dense(16,activation='relu'))
model.add(L.Dropout(0.4))
model.add(L.Dense(2,activation='softmax'))

model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 150, 16)           142336    
                                                                 
 bidirectional_6 (Bidirectio  (None, 150, 512)         559104    
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 150, 512)         1574912   
 nal)                                                            
                                                                 
 global_max_pooling1d_3 (Glo  (None, 512)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                      

### Huấn luyện

In [53]:
EPOCHS = 10
BATCH_SIZE = 32
URL_TO_SAVE_MODEL = '/content/drive/MyDrive/NLP_Food_review_classifier/Final_Chu_Tien_Dat/model/model.h5'


history = model.fit(X_train, y_train, epochs=EPOCHS, verbose=True, validation_data=(X_test, y_test), batch_size=BATCH_SIZE)

model.save(URL_TO_SAVE_MODEL)

213/213 [==============================] - 584s 3s/step - loss: 0.4611 - accuracy: 0.8186 - val_loss: 0.2791 - val_accuracy: 0.9061


# Đánh giá mô hình

In [62]:
y_test_pred = model.predict(X_test).argmax(axis=1)
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred)
auc = auc(fpr, tpr)
print("AUC = ", auc)

71/71 [==============================] - 31s 435ms/step


TypeError: ignored

### Xử lý dữ liệu dự đoán

In [59]:
comment_pred = df_test['Comment'].apply(lambda x: preprocess(x))
X_pred = tokenizer.texts_to_sequences(comment_pred)
X_pred = pad_sequences(X_pred, padding='post', maxlen=maxlen)

### Dự đoán

In [ ]:
y_pred = model.predict(X_pred).argmax(axis=1)
df_test['Rating'] = y_pred
df_test[['RevId', 'Rating']].to_csv('/content/drive/MyDrive/NLP_Food_review_classifier/Final_Chu_Tien_Dat/result/result.csv', index = False)